In [ ]:
import pandas as pd
import requests
import urllib.request
from bs4 import BeautifulSoup
import nltk
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import string
from textblob import TextBlob
import csv
import numpy as np

In [ ]:
!pip install openpyxl

In [ ]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_excel('/content/drive/MyDrive/20211030 Test Assignment/Input.xlsx')

In [ ]:
df.head()

,URL_ID,URL
0,123.0,https://insights.blackcoffer.com/rise-of-telem...
1,321.0,https://insights.blackcoffer.com/rise-of-e-hea...
2,2345.0,https://insights.blackcoffer.com/rise-of-e-hea...
3,4321.0,https://insights.blackcoffer.com/rise-of-telem...
4,432.0,https://insights.blackcoffer.com/rise-of-telem...


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 114 entries, 0 to 113
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   URL_ID  114 non-null    float64
 1   URL     114 non-null    object 
dtypes: float64(1), object(1)
memory usage: 1.9+ KB


In [ ]:
links = [url for url in df['URL'][:61]]



In [ ]:
# Loop through each URL in the links

for url in links:

    # Send a GET request to the URL
    response = requests.get(url)

    # Check status code 200
    if response.status_code == 200:

        # Parse the HTML content of the page using BeautifulSoup
        soup = BeautifulSoup(response.text, 'html.parser')

        # Extract the data you need from the parsed HTML
        paragraphs = soup.find(attrs={'class': 'td-post-content'})

        # Convert the ResultSet to a list to avoid 'NoneType' issue
        paragraphs_list = list(paragraphs)

        # Loop through each paragraph in the list
        for i in range(len(paragraphs_list)):
            # Check if the paragraph is not None before applying replace
            if paragraphs_list[i] is not None:
                paragraphs_list[i] = paragraphs_list[i].text.replace('\n', '')

                # Print or process the extracted data
                print(paragraphs_list[i])
    else:
        print(f"Failed to retrieve content from {url}. Status code: {response.status_code}")



Telemedicine, the use of technology to diagnose and treat patients remotely, has been rising in recent years. With the advent of high-speed internet and improved video conferencing tools, healthcare providers are increasingly turning to telemedicine to provide care to patients in remote or underserved areas.

Telemedicine, using technology to provide healthcare services remotely, has recently gained popularity. With advancements in communication and medical technology, it has become increasingly possible for doctors and patients to connect and interact from anywhere in the world. This has led to the rise of telemedicine, which has the potential to revolutionize the way healthcare is delivered.#Telemedicine

The increasing focus on preventative healthcare has also driven the rise of telemedicine. As more and more people become aware of the importance of staying healthy, they are looking for ways to prevent illness and maintain their health. Telemedicine allows people to monitor their h

In [ ]:
sentences = []

for i in paragraphs_list:
  sentences.append(len(sent_tokenize( i )))



In [ ]:
words = []
for i in paragraphs_list:
  words.append(len(word_tokenize(i)))

In [ ]:
stopWordsFile =     '/content/drive/MyDrive/20211030 Test Assignment/StopWords/StopWords_Generic.txt'
positiveWordsFile = '/content/drive/MyDrive/20211030 Test Assignment/MasterDictionary/positive-words.txt'
negativeWordsFile = '/content/drive/MyDrive/20211030 Test Assignment/MasterDictionary/negative-words.txt'

In [ ]:
#Loading stop words dictionary for removing stop words

with open(stopWordsFile ,'r') as stop_words:
    stopWords = stop_words.read().lower()
stopWordList = stopWords.split('\n')

In [ ]:
# Loading positive words
with open(positiveWordsFile,'r') as posfile:
    positivewords=posfile.read().lower()
positiveWordList=positivewords.split('\n')

In [ ]:
# Loading negative words
with open(negativeWordsFile ,'r',encoding="ISO-8859-1" ) as negfile:
    negativeword=negfile.read().lower()
negativeWordList=negativeword.split('\n')

In [ ]:
cleaned_articles = [' ']*len(paragraphs_list)

In [ ]:
for i in range(len(paragraphs_list)):
  for w in stopWordList:
    cleaned_articles[i]= paragraphs_list[i].replace(' '+w+' ',' ').replace('?','').replace('.','').replace(',','').replace('!','')


In [ ]:
words_cleaned = []
for article in cleaned_articles:
  words_cleaned.append(len(word_tokenize(article)))

In [ ]:
positive_score = [0]*len(paragraphs_list)
for i in range(len(paragraphs_list)):
  for word in positiveWordList:
    for letter in cleaned_articles[i].lower().split(' '):
      if letter==word:
        positive_score[i]+=1

In [ ]:
negative_score = [0]*len(paragraphs_list)
for i in range(len(paragraphs_list)):
  for word in negativeWordList:
    for letter in cleaned_articles[i].upper().split(' '):
      if letter==word:
        negative_score[i]+=1

In [ ]:
words_cleaned = np.array(words_cleaned)
sentences = np.array(sentences)

In [ ]:
df1 = df[:47]

In [ ]:
df1['POSITIVE SCORE'] = positive_score
df1['NEGATIVE SCORE'] = negative_score

In [ ]:
df1['POLARITY SCORE'] = (df1['POSITIVE SCORE']-df1['NEGATIVE SCORE'])/ ((df1['POSITIVE SCORE'] +df1['NEGATIVE SCORE']) + 0.000001)
df1['SUBJECTIVITY SCORE'] = (df1['POSITIVE SCORE'] + df1['NEGATIVE SCORE'])/( (words_cleaned) + 0.000001)
df1['AVG SENTENCE LENGTH'] = np.array(words)/np.array(sentences)

In [ ]:
complex_words = []
sylabble_counts = []

for article in paragraphs_list:
  sylabble_count=0
  d=article.split()
  ans=0
  for word in d:
    count=0
    for i in range(len(word)):
      if(word[i]=='a' or word[i]=='e' or word[i] =='i' or word[i] == 'o' or word[i] == 'u'):
           count+=1
#            print(words[i])
      if(i==len(word)-2 and (word[i]=='e' and word[i+1]=='d')):
        count-=1;
      if(i==len(word)-2 and (word[i]=='e' and word[i]=='s')):
        count-=1;
    sylabble_count+=count
    if(count>2):
        ans+=1
  sylabble_counts.append(sylabble_count)
  complex_words.append(ans)

In [ ]:
df1['PERCENTAGE OF COMPLEX WORDS'] = np.array(complex_words)/np.array(words)

In [ ]:
df1['FOG INDEX'] = 0.4 * (df1['AVG SENTENCE LENGTH'] + df1['PERCENTAGE OF COMPLEX WORDS'])

In [ ]:
df1['AVG NUMBER OF WORDS PER SENTENCES'] = df1['AVG SENTENCE LENGTH']

In [ ]:
df1['COMPLEX WORD COUNT'] = complex_words

In [ ]:
df1['WORD COUNT'] = words

In [ ]:
total_characters = []
for article in paragraphs_list:
  characters = 0
  for word in str(paragraphs_list).split():
    characters+=len(word)
  total_characters.append(characters)

In [ ]:
personal_nouns = []
personal_noun =['I', 'we','my', 'ours','and' 'us','My','We','Ours','Us','And']
for article in paragraphs_list:
  ans=0
  for word in article:
    if word in personal_noun:
      ans+=1
  personal_nouns.append(ans)

In [ ]:
df1['PERSONAL PRONOUN'] = personal_nouns

In [ ]:
df1['AVG WORD LENGTH'] = np.array(total_characters)/np.array(words)

In [ ]:
df1.isna().sum()

URL_ID                                0
URL                                   0
POSITIVE SCORE                        0
NEGATIVE SCORE                        0
POLARITY SCORE                        0
SUBJECTIVITY SCORE                    0
AVG SENTENCE LENGTH                  25
PERCENTAGE OF COMPLEX WORDS          25
FOG INDEX                            25
AVG NUMBER OF WORDS PER SENTENCES    25
COMPLEX WORD COUNT                    0
WORD COUNT                            0
PERSONAL PRONOUN                      0
AVG WORD LENGTH                       0
dtype: int64

In [ ]:
df1['AVG SENTENCE LENGTH'].fillna(df1['AVG SENTENCE LENGTH'].mean(),inplace=True)
df1['PERCENTAGE OF COMPLEX WORDS'].fillna(df1['PERCENTAGE OF COMPLEX WORDS'].mean(),inplace=True)
df1['FOG INDEX'].fillna(df1['FOG INDEX'].mean(),inplace=True)
df1['AVG NUMBER OF WORDS PER SENTENCES'].fillna(df1['AVG NUMBER OF WORDS PER SENTENCES'].mean(),inplace=True)

In [ ]:
df1.head(20)

,URL_ID,URL,POSITIVE SCORE,NEGATIVE SCORE,POLARITY SCORE,SUBJECTIVITY SCORE,AVG SENTENCE LENGTH,PERCENTAGE OF COMPLEX WORDS,FOG INDEX,AVG NUMBER OF WORDS PER SENTENCES,COMPLEX WORD COUNT,WORD COUNT,PERSONAL PRONOUN,AVG WORD LENGTH
0,123.0,https://insights.blackcoffer.com/rise-of-telem...,1,1,0.000000,2.000000e+06,16.906494,0.154878,6.824549,16.906494,0,0,0,inf
1,321.0,https://insights.blackcoffer.com/rise-of-e-hea...,3,0,1.000000,6.818182e-02,9.800000,0.183673,3.993469,9.800000,9,49,0,110.306122
2,2345.0,https://insights.blackcoffer.com/rise-of-e-hea...,1,1,0.000000,2.000000e+06,16.906494,0.154878,6.824549,16.906494,0,0,0,inf
3,4321.0,https://insights.blackcoffer.com/rise-of-telem...,1,0,0.999999,2.272727e-02,12.000000,0.062500,4.825000,12.000000,3,48,0,112.604167
4,432.0,https://insights.blackcoffer.com/rise-of-telem...,1,1,0.000000,2.000000e+06,16.906494,0.154878,6.824549,16.906494,0,0,0,inf
5,2893.8,https://insights.blackcoffer.com/rise-of-chatb...,0,0,0.000000,0.000000e+00,17.500000,0.157143,7.062857,17.500000,11,70,0,77.214286
6,3355.6,https://insights.blackcoffer.com/rise-of-e-hea...,1,1,0.000000,2.000000e+06,16.906494,0.154878,6.824549,16.906494,0,0,0,inf
7,3817.4,https://insights.blackcoffer.com/how-does-mark...,1,0,0.999999,1.587302e-02,14.200000,0.112676,5.725070,14.200000,8,71,0,76.126761
8,4279.2,https://insights.blackcoffer.com/how-advertise...,1,1,0.000000,2.000000e+06,16.906494,0.154878,6.824549,16.906494,0,0,0,inf
9,4741.0,https://insights.blackcoffer.com/negative-effe...,2,0,1.000000,5.714286e-02,18.500000,0.216216,7.486486,18.500000,8,37,0,146.081081


In [ ]:

df1.to_csv('output.csv', index=False)